# Off-Platform Project: Classifying Tweets

In this off-platform project, you will use a Naive Bayes Classifier to find patterns in real tweets. We've given you three files: `new_york.json`, `london.json`, and `paris.json`. These three files contain tweets that we gathered from those locations.

The goal is to create a classification algorithm that can classify any tweet (or sentence) and predict whether that sentence came from New York, London, or Paris.

# Investigate the Data

To begin, let's take a look at the data. We've imported `new_york.json` and printed the following information:
* The number of tweets.
* The columns, or features, of a tweet.
* The text of the 12th tweet in the New York dataset.

Run the cell below to see the results.

In [1]:
import pandas as pd

new_york_tweets = pd.read_json("new_york.json", lines=True)
print(len(new_york_tweets))
print(new_york_tweets.columns)
print(new_york_tweets.loc[12]["text"])

4723
Index([u'contributors', u'coordinates', u'created_at', u'display_text_range',
       u'entities', u'extended_entities', u'extended_tweet', u'favorite_count',
       u'favorited', u'filter_level', u'geo', u'id', u'id_str',
       u'in_reply_to_screen_name', u'in_reply_to_status_id',
       u'in_reply_to_status_id_str', u'in_reply_to_user_id',
       u'in_reply_to_user_id_str', u'is_quote_status', u'lang', u'place',
       u'possibly_sensitive', u'quote_count', u'quoted_status',
       u'quoted_status_id', u'quoted_status_id_str',
       u'quoted_status_permalink', u'reply_count', u'retweet_count',
       u'retweeted', u'source', u'text', u'timestamp_ms', u'truncated',
       u'user', u'withheld_in_countries'],
      dtype='object')
Be best #ThursdayThoughts


In the code block below, load the London and Paris tweets into DataFrames named `london_tweets` and `paris_tweets`.

How many London tweets are there? How many Paris ones are there?

In [9]:
london_tweets = pd.read_json("london.json", lines = True)
print("Length of London tweets:", len(london_tweets))
print("Columns of the London tweets:", london_tweets.columns)


paris_tweets = pd.read_json("paris.json", lines = True)
print("Length of Paris tweets:", len(paris_tweets))
print("Columns of the Paris tweets:", paris_tweets.columns)

('Length of London tweets:', 5341)
('Columns of the London tweets:', Index([u'contributors', u'coordinates', u'created_at', u'display_text_range',
       u'entities', u'extended_entities', u'extended_tweet', u'favorite_count',
       u'favorited', u'filter_level', u'geo', u'id', u'id_str',
       u'in_reply_to_screen_name', u'in_reply_to_status_id',
       u'in_reply_to_status_id_str', u'in_reply_to_user_id',
       u'in_reply_to_user_id_str', u'is_quote_status', u'lang', u'place',
       u'possibly_sensitive', u'quote_count', u'quoted_status',
       u'quoted_status_id', u'quoted_status_id_str',
       u'quoted_status_permalink', u'reply_count', u'retweet_count',
       u'retweeted', u'source', u'text', u'timestamp_ms', u'truncated',
       u'user'],
      dtype='object'))
('Length of Paris tweets:', 2510)
('Columns of the Paris tweets:', Index([u'contributors', u'coordinates', u'created_at', u'display_text_range',
       u'entities', u'extended_entities', u'extended_tweet', u'favorit

# Classifying using language: Naive Bayes Classifier

We're going to create a Naive Bayes Classifier! Let's begin by looking at the way language is used differently in these three locations. Let's grab the text of all of the tweets and make it one big list. In the code block below, we've created a list of all the New York tweets. Do the same for `london_tweets` and `paris_tweets`.

Then combine all three into a variable named `all_tweets` by using the `+` operator. For example, `all_tweets = new_york_text + london_text + ...`

Let's also make the labels associated with those tweets. `0` represents a New York tweet, `1`  represents a London tweet, and `2` represents a Paris tweet. Finish the definition of `labels`.

In [10]:
new_york_text = new_york_tweets["text"].tolist()
london_text = london_tweets["text"].tolist()
paris_text = paris_tweets["text"].tolist()

all_tweets = new_york_text + london_text + paris_text
labels = [0] * len(new_york_text) + [1] * len(london_text) + [2] * len(paris_text)

# Making a Training and Test Set

We can now break our data into a training set and a test set. We'll use scikit-learn's `train_test_split` function to do this split. This function takes two required parameters: It takes the data, followed by the labels. Set the optional parameter `test_size` to be `0.2`. Finally, set the optional parameter `random_state` to `1`. This will make it so your data is split in the same way as the data in our solution code. 

Remember, this function returns 4 items in this order:
1. The training data
2. The testing data
3. The training labels
4. The testing labels

Store the results in variables named `train_data`, `test_data`, `train_labels`, and `test_labels`.

Print the length of `train_data` and the length of `test_data`.

In [11]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(all_tweets, labels, 
                                                                    test_size = 0.2,
                                                                    random_state = 1)


# Making the Count Vectors

To use a Naive Bayes Classifier, we need to transform our lists of words into count vectors. Recall that this changes the sentence `"I love New York, New York"` into a list that contains:

* Two `1`s because the words `"I"` and `"love"` each appear once.
* Two `2`s because the words `"New"` and `"York"` each appear twice.
* Many `0`s because every other word in the training set didn't appear at all.

To start, create a `CountVectorizer` named `counter`.

Next, call the `.fit()` method using `train_data` as a parameter. This teaches the counter our vocabulary.

Finally, let's transform `train_data` and `test_data` into Count Vectors. Call `counter`'s `.transform()` method using `train_data` as a parameter and store the result in `train_counts`. Do the same for `test_data` and store the result in `test_counts`.

Print `train_data[3]` and `train_counts[3]` to see what a tweet looks like as a Count Vector.

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

counter = CountVectorizer()
counter.fit(train_data)

train_counts = counter.transform(train_data)
test_counts = counter.transform(test_data)

print(train_data[3], train_counts[3])


(u'saying bye is hard. Especially when youre saying bye to comfort.', <1x32591 sparse matrix of type '<type 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>)


# Train and Test the Naive Bayes Classifier

We now have the inputs to our classifier. Let's use the CountVectors to train and test the Naive Bayes Classifier!

First, make a `MultinomialNB` named `classifier`.

Next, call `classifier`'s `.fit()` method. This method takes two parameters &mdash; the training data and the training labels. `train_counts` contains the training data and `train_labels` containts the labels for that data.

Calling `.fit()` calculates all of the probabilities used in Bayes Theorem. The model is now ready to quickly predict the location of a new tweet. 

Finally, let's test our model. `classifier`'s `.predict()` method using `test_counts` as a parameter. Store the results in a variable named `predictions`.

In [15]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(train_counts, train_labels)
predictions = classifier.predict(test_counts)


# Evaluating Your Model

Now that the classifier has made its predictions, let's see how well it did. Let's look at two different ways to do this. First, call scikit-learn's `accuracy_score` function. This function should take two parameters &mdash;  the `test_labels` and your `predictions`. Print the results. This prints the percentage of tweets in the test set that the classifier correctly classified.



In [17]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_labels, predictions)
print(accuracy)


0.6779324055666004


The other way you can evaluate your model is by looking at the **confusion matrix**. A confusion matrix is a table that describes how your classifier made its predictions. For example, if there were two labels, A and B, a confusion matrix might look like this:

```
9 1
3 5
```

In this example, the first row shows how the classifier classified the true A's. It guessed that 9 of them were A's and 1 of them was a B. The second row shows how the classifier did on the true B's. It guessed that 3 of them were A's and 5 of them were B's.

For our project using tweets, there were three classes &mdash; `0` for New York, `1` for London, and `2` for Paris. You can see the confustion matrix by printing the result of the `confusion_matrix` function using `test_labels` and `predictions` as parameters.

In [18]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(test_labels, predictions))


[[541 404  28]
 [203 824  34]
 [ 38 103 340]]


# Test Your Own Tweet

Nice work! The confusion matrix should line up with your intuition. The classifier predicts tweets that were actually from New York as either New York tweets or London tweets, but almost never Paris tweets. Similarly, the classifier rarely misclassifies the tweets that were actually from Paris. Tweets coming from two English speaking countries are harder to distinguish than tweets in different languages.

Now it's your chance to write a tweet and see how the classifier works! Create a string and store it in a variable named `tweet`. 

Call `counter`'s `.transform()` method using `[tweet]` as a parameter. Save the result as `tweet_counts`. Notice that your variable has to be in an array &mdash; `.transform()` can't take just a string, it must be a list. 

Finally, pass `tweet_counts` as parameter to `classifier`'s `.predict()` method. Print the result. This should give you the prediction for the tweet. Remember a `0` represents New York, a `1` represents London, and a `2` represents Paris. Can you write different tweets that the classifier predicts as being from New York, London, and Paris?

In [19]:
tweet = "Guys in this city are fucking handsome"
tweet_counts = counter.transform([tweet])
print(classifier.predict(tweet_counts))

[0]


My tweet is classified as a tweet from New York

# Use Cross Validation to increase the robustness of the Naive Bayes Classifier

In [32]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import SCORERS

all_counts = counter.transform(all_tweets) #tcreate a counter object without splitting the data
scores1 = cross_val_score(classifier, all_counts, labels, cv=5, scoring = "accuracy")

print("All accuracy scores from folds:", scores1)
print("Mean accuracy score from the cross validation:", scores1.mean())

('All accuracy scores from folds:', array([0.64864865, 0.66202783, 0.66918489, 0.67064439, 0.67501989]))
('Mean accuracy score from the cross validation:', 0.6651051304677045)


The average accuracy score of  the whole dataset is 66%, which concludes that our train and test split worked well in terms of accuracy score.